In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Biogeme
import biogeme.database as db
import biogeme.biogeme as bio
from biogeme.expressions import Beta, Variable, log, exp
from biogeme.results_processing import get_pandas_estimated_parameters

# Random Forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

## Data exploration and analysis preparation

#### Reading the data and converting them into a Dataframe 

In [6]:
# Load all dataframes (using data_loader module)

from data_loader import load_dataset as load_all

other_road_users, sensors = load_all()
print('Loaded', len(other_road_users), 'other dataframes and', len(sensors), 'sensors dataframes')

# Display sample keys
print('Sample other_road_users keys:', list(other_road_users.keys())[:10])
print('Sample sensors keys:', list(sensors.keys())[:10])

Loaded 12 other dataframes and 12 sensors dataframes
Sample other_road_users keys: ['ebike_subject_a', 'ebike_subject_b', 'escooter_subject_b', 'escooter_subject_c', 'ebike_subject_e', 'escooter_subject_e', 'ebike_subject_g', 'escooter_subject_g', 'ebike_subject_h', 'escooter_subject_h']
Sample sensors keys: ['ebike_subject_a', 'ebike_subject_b', 'escooter_subject_b', 'escooter_subject_c', 'ebike_subject_e', 'escooter_subject_e', 'ebike_subject_g', 'escooter_subject_g', 'ebike_subject_h', 'escooter_subject_h']


Beware : other_road_users is a dictionnary of dataframes, key = MMV_subject_x , value is associated data

In [4]:
# See the columns of other_road_users dataframes
print('Columns of other_road_users:', list(other_road_users['ebike_subject_a'].columns) )
# See the columns of sensors dataframes
print( 'Columns of sensors:', list(sensors['ebike_subject_b'].columns) )

Columns of other_road_users: ['frame_index', 'track_id', 'class_name', 'angle', 'distance', 'x', 'y', 'x_inter', 'y_inter', 'x_inter_rts', 'y_inter_rts', 'vx_rts', 'vy_rts', 'interpolated', 'corrected_class']
Columns of sensors: ['timestamp', 'cts_gopro', 'date_gopro', 'GPS (Lat.) [deg]', 'GPS (Long.) [deg]', 'GPS (Alt.) [m]', 'GPS (2D speed) [m/s]', 'precision', 'cts_gyro', 'date_gyro', 'Gyroscope (z) [rad/s]', 'Gyroscope (x) [rad/s]', 'Gyroscope (y) [rad/s]', 'cts_acc', 'date_acc', 'Accelerometer (z) [m/s²]', 'Accelerometer (x) [m/s²]', 'Accelerometer (y) [m/s²]', 'Accelerometer (x) Filtered [m/s²]', 'Accelerometer (y) Filtered [m/s²]', 'Accelerometer (z) Filtered [m/s²]', 'Filtered velocity [m/s]', 'bearing', 'frame_index']


In [5]:
# Remove all the missing data in all the dataframes (Ghislain ajoute cette cellule après avoir chargé les dataframes)

for key in other_road_users:
    other_road_users[key] = other_road_users[key].dropna()
for key in sensors:
    sensors[key] = sensors[key].dropna()

## Merging, feature creation, and broadcasting of static questionnaire data.

In [8]:
# -----------------------------------------------------------------------------
# 1. SETUP: ASSUMING DATA IS LOADED
# -----------------------------------------------------------------------------

# List to store processed dataframes before final concatenation
processed_frames = []

# -----------------------------------------------------------------------------
# 2. ITERATE THROUGH SUBJECTS
# -----------------------------------------------------------------------------
# We iterate through the keys of the sensors dictionary
for key in sensors.keys():
    
    # A. Extract DataFrames
    # ---------------------
    if key not in other_road_users:
        print(f"Warning: No user data found for {key}. Skipping.")
        continue
        
    df_sensor = sensors[key].copy()
    df_user = other_road_users[key].copy()
    
    # Sort by timestamp/frame just to be safe
    df_sensor = df_sensor.sort_values(by='timestamp')
    
    # B. Filter "Other Road Users" (Keep Closest Only)
    # ------------------------------------------------
    # We only want one interaction per timestamp for the simple regression model.
    # We sort by frame_index and distance, then keep the first (closest).
    df_user_sorted = df_user.sort_values(by=['frame_index', 'distance'])
    df_user_closest = df_user_sorted.drop_duplicates(subset=['frame_index'], keep='first')
    
    # C. Merge
    # --------
    # Left merge: We keep all sensor data (rider state), 
    # and attach pedestrian data where available.
    df_merged = pd.merge(df_sensor, df_user_closest, on='frame_index', how='left')
    
    # D. Metadata Parsing (From Key)
    # ------------------------------
    # Keys are like 'ebike_subject_a' or 'escooter_subject_b'
    # We split the string to get vehicle type and subject ID
    parts = key.split('_') # ['ebike', 'subject', 'a']
    
    vehicle_type = parts[0] # 'ebike' or 'escooter'
    subject_id = f"{parts[1]}_{parts[2]}" # 'subject_a'
    
    df_merged['vehicle_type'] = vehicle_type
    df_merged['subject_id'] = subject_id
    
    # E. Feature Engineering (Physics)
    # --------------------------------
    
    # 1. Handle Missing Detection Data
    # If no pedestrian detected, distance is infinite (safe). 
    # We use 50m as a "max relevant distance".
    df_merged['distance'] = df_merged['distance'].fillna(50.0)
    df_merged['angle'] = df_merged['angle'].fillna(0.0)
    
    # 2. Closing Speed (m/s)
    # 'vx_rts' is usually the relative velocity in X. 
    # If x decreases (getting closer), vx is negative.
    # Closing speed = -1 * Relative Velocity (so positive means closing in)
    df_merged['vx_rts'] = df_merged['vx_rts'].fillna(0.0)
    df_merged['closing_speed'] = -1 * df_merged['vx_rts']
    
    # 3. Inverse Time-To-Collision (iTTC)
    # iTTC = Closing Speed / Distance
    # We use Inverse because TTC is infinite when safe (causes math errors).
    # iTTC = 0 means safe/infinite time. High value means danger.
    # We only calculate if closing_speed > 0 (getting closer).
    condition_danger = (df_merged['closing_speed'] > 0) & (df_merged['distance'] > 0.1)
    
    df_merged['iTTC'] = np.where(
        condition_danger,
        df_merged['closing_speed'] / df_merged['distance'],
        0.0
    )
    
    # F. Lagged Variables (Auto-regression)
    # -------------------------------------
    # We calculate this HERE, inside the loop, so the last second of 
    # Subject A doesn't become the "previous speed" for Subject B.
    
    # Predictor: Speed at t-1
    df_merged['v_t_minus_1'] = df_merged['Filtered velocity [m/s]'].shift(1)
    
    # Target: Acceleration at t (already exists), but let's ensure units are clean
    # 'Accelerometer (x) Filtered' is our Y
    
    # Drop the first row of this subject (NaN due to shift)
    df_merged = df_merged.dropna(subset=['v_t_minus_1'])
    
    # Add to list
    processed_frames.append(df_merged)

# -----------------------------------------------------------------------------
# 3. CONCATENATE FINAL DATASET
# -----------------------------------------------------------------------------
final_dataset = pd.concat(processed_frames, ignore_index=True)

# -----------------------------------------------------------------------------
# 4. (Optional) MAPPING STATIC QUESTIONNAIRES
# -----------------------------------------------------------------------------
# If you have the scores, map them now using the 'subject_id' column created in Step D.
# Example dictionary (You need to fill this with real values from the questionnaires):
cbq_scores = {
    'subject_a': 1.2, 'subject_b': 3.5, 'subject_c': 0.8,
    'subject_e': 2.1, 'subject_g': 1.5, 'subject_h': 4.0
}

# Map new column
final_dataset['CBQ_Violations'] = final_dataset['subject_id'].map(cbq_scores)

# -----------------------------------------------------------------------------
# 5. INSPECT
# -----------------------------------------------------------------------------
print(f"Total rows: {len(final_dataset)}")
print("Columns:", final_dataset.columns.tolist())
print(final_dataset[['subject_id', 'vehicle_type', 'v_t_minus_1', 'distance', 'iTTC']].head())

Total rows: 4651
Columns: ['timestamp', 'cts_gopro', 'date_gopro', 'GPS (Lat.) [deg]', 'GPS (Long.) [deg]', 'GPS (Alt.) [m]', 'GPS (2D speed) [m/s]', 'precision', 'cts_gyro', 'date_gyro', 'Gyroscope (z) [rad/s]', 'Gyroscope (x) [rad/s]', 'Gyroscope (y) [rad/s]', 'cts_acc', 'date_acc', 'Accelerometer (z) [m/s²]', 'Accelerometer (x) [m/s²]', 'Accelerometer (y) [m/s²]', 'Accelerometer (x) Filtered [m/s²]', 'Accelerometer (y) Filtered [m/s²]', 'Accelerometer (z) Filtered [m/s²]', 'Filtered velocity [m/s]', 'bearing', 'frame_index', 'track_id', 'class_name', 'angle', 'distance', 'x', 'y', 'x_inter', 'y_inter', 'x_inter_rts', 'y_inter_rts', 'vx_rts', 'vy_rts', 'interpolated', 'corrected_class', 'vehicle_type', 'subject_id', 'closing_speed', 'iTTC', 'v_t_minus_1', 'CBQ_Violations']
  subject_id vehicle_type  v_t_minus_1  distance  iTTC
0  subject_a        ebike     2.026448      50.0   0.0
1  subject_a        ebike     2.154997      50.0   0.0
2  subject_a        ebike     2.280301      50.0 

## Model 1 : Linear regression model

In [ ]:
# 1. Prepare Data for Biogeme
# Biogeme does not handle spaces/parentheses in column names well.
# We rename the columns from the 'final_dataset' created in the previous step.
df_bio = final_dataset.rename(columns={
    'Accelerometer (x) Filtered [m/s²]': 'Acc_X',
    'Filtered velocity [m/s]': 'Speed',
    'v_t_minus_1': 'Speed_Lag',
    'distance': 'Distance',
    'iTTC': 'Inv_TTC'
})

print("Columns for Biogeme:", df_bio.columns.tolist())

df_bio = df_bio[['Acc_X', 'Speed_Lag', 'Distance', 'Inv_TTC']]

# 2. Initialize Biogeme Database
database = db.Database("micromobility_linear_model", df_bio)

# 3. Define Parameters to Estimate (Name, Start Value, LowerBound, UpperBound, Fixed)
B_Intercept = Beta('B_Intercept', 0, None, None, 0)
B_SpeedLag  = Beta('B_SpeedLag', 0, None, None, 0)  # Autoregressive term
B_Distance  = Beta('B_Distance', 0, None, None, 0)  # Influence of proximity
B_iTTC      = Beta('B_iTTC', 0, None, None, 0)      # Influence of collision risk
B_Sigma     = Beta('B_Sigma', 1, None, None, 0)     # Standard deviation of error

# 4. Define Variables from Database
Acc_X     = Variable('Acc_X')
Speed_Lag = Variable('Speed_Lag')
Distance  = Variable('Distance')
Inv_TTC   = Variable('Inv_TTC')

# 5. Model Specification (Linear Regression)
# Model: Acc(t) = B0 + B1*Speed(t-1) + B2*Distance + B3*iTTC + epsilon
Utility = B_Intercept + \
             (B_SpeedLag * Speed_Lag) + \
             (B_Distance * Distance) + \
             (B_iTTC * Inv_TTC)

# 6. Likelihood Function (Normal Distribution)
# We maximize the likelihood that the errors are normally distributed.
likelihood = -0.5 * log(2 * 3.14159) - log(B_Sigma) - \
             0.5 * ((Acc_X - Utility) / B_Sigma)**2

# 7. Estimate
biogeme_obj = bio.BIOGEME(database, likelihood)
biogeme_obj.modelName = "micromobility_linear_regression"
results = biogeme_obj.estimate()

# 8. Print Results
print(results.short_summary())
print(get_pandas_estimated_parameters(estimation_results=results))

Columns for Biogeme: ['timestamp', 'cts_gopro', 'date_gopro', 'GPS (Lat.) [deg]', 'GPS (Long.) [deg]', 'GPS (Alt.) [m]', 'GPS (2D speed) [m/s]', 'precision', 'cts_gyro', 'date_gyro', 'Gyroscope (z) [rad/s]', 'Gyroscope (x) [rad/s]', 'Gyroscope (y) [rad/s]', 'cts_acc', 'date_acc', 'Accelerometer (z) [m/s²]', 'Accelerometer (x) [m/s²]', 'Accelerometer (y) [m/s²]', 'Acc_X', 'Accelerometer (y) Filtered [m/s²]', 'Accelerometer (z) Filtered [m/s²]', 'Speed', 'bearing', 'frame_index', 'track_id', 'class_name', 'angle', 'Distance', 'x', 'y', 'x_inter', 'y_inter', 'x_inter_rts', 'y_inter_rts', 'vx_rts', 'vy_rts', 'interpolated', 'corrected_class', 'vehicle_type', 'subject_id', 'closing_speed', 'Inv_TTC', 'Speed_Lag', 'CBQ_Violations']


/tmp/ipykernel_4424/1561534422.py:52: DeprecationWarning: 'modelName' is deprecated. Please use 'model_name' instead.
  biogeme_obj.modelName = "micromobility_linear_regression"


Results for model micromobility_linear_regression
Nbr of parameters:		5
Sample size:			4651
Excluded data:			0
Final log likelihood:		-5077.235
Akaike Information Criterion:	10164.47
Bayesian Information Criterion:	10196.69

          Name     Value  Robust std err.  Robust t-stat.  Robust p-value
0      B_Sigma  0.720872         0.010790       66.807109    0.000000e+00
1  B_Intercept  0.231235         0.031923        7.243598    4.369838e-13
2   B_SpeedLag -0.027394         0.008554       -3.202483    1.362481e-03
3   B_Distance  0.001065         0.000491        2.168825    3.009594e-02
4       B_iTTC  0.072894         0.066573        1.094950    2.735386e-01


## Model 2 : Radom forest

In [23]:
# 1. Prepare Features (X) and Target (y)
# We use the raw 'final_dataset' from the previous step.

# Create a numeric dummy variable for vehicle type
# 1 if E-scooter, 0 if E-bike
final_dataset['is_escooter'] = (final_dataset['vehicle_type'] == 'escooter').astype(int)

features = [
    'v_t_minus_1',   # Lagged Speed (State)
    'distance',      # Distance to pedestrian (Context)
    'angle',         # Angle to pedestrian (Context)
    'iTTC',          # Inverse Time to Collision (Interaction)
    'is_escooter'    # Vehicle Type (Control)
]

print("Columns in final_dataset:", final_dataset.columns.tolist())

target = 'Accelerometer (x) Filtered [m/s²]'

# Drop rows with NaN in these specific columns to ensure clean training
df_rf = final_dataset.dropna(subset=features + [target])

X = df_rf[features]
y = df_rf[target]

# 2. Train/Test Split (80% Train, 20% Test)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 3. Initialize Random Forest
rf_model = RandomForestRegressor(
    n_estimators=100,  # Number of trees
    max_depth=10,      # Limit depth to prevent overfitting
    random_state=42,
    n_jobs=-1          # Parallel processing
)

# 4. Train the Model
rf_model.fit(X_train, y_train)

# 5. Evaluate
y_pred = rf_model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("--- Random Forest Results ---")
print(f"Mean Squared Error: {mse:.4f}")
print(f"R^2 Score: {r2:.4f}")

# 6. Feature Importance (Interpretation)
importances = rf_model.feature_importances_
for name, importance in zip(features, importances):
    print(f"Feature: {name}, Importance: {importance:.4f}")

Columns in final_dataset: ['timestamp', 'cts_gopro', 'date_gopro', 'GPS (Lat.) [deg]', 'GPS (Long.) [deg]', 'GPS (Alt.) [m]', 'GPS (2D speed) [m/s]', 'precision', 'cts_gyro', 'date_gyro', 'Gyroscope (z) [rad/s]', 'Gyroscope (x) [rad/s]', 'Gyroscope (y) [rad/s]', 'cts_acc', 'date_acc', 'Accelerometer (z) [m/s²]', 'Accelerometer (x) [m/s²]', 'Accelerometer (y) [m/s²]', 'Accelerometer (x) Filtered [m/s²]', 'Accelerometer (y) Filtered [m/s²]', 'Accelerometer (z) Filtered [m/s²]', 'Filtered velocity [m/s]', 'bearing', 'frame_index', 'track_id', 'class_name', 'angle', 'distance', 'x', 'y', 'x_inter', 'y_inter', 'x_inter_rts', 'y_inter_rts', 'vx_rts', 'vy_rts', 'interpolated', 'corrected_class', 'vehicle_type', 'subject_id', 'closing_speed', 'iTTC', 'v_t_minus_1', 'CBQ_Violations', 'is_escooter']
--- Random Forest Results ---
Mean Squared Error: 0.3941
R^2 Score: 0.1693
Feature: v_t_minus_1, Importance: 0.4552
Feature: distance, Importance: 0.1457
Feature: angle, Importance: 0.1631
Feature: i